Coding a Neural Network from Scratch

Activation Function

In [4]:
import numpy as np

def sigmoid(x):
    # Our activation function: f(x) = 1 / (1 + e^(-x))
    return 1/ (1 + np.exp(-x))

Creating a Neuron

In [5]:
class Neuron:

    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
    
    def feed_forward(self, inputs):
        # Weight inputs, add bias, then use the activation function
        total = np.dot(self.weight, inputs) + self.bias
        return sigmoid(total)

Implementing a Neuron

In [6]:
weights = np.array([0,1]) # w1 = 0, w2 = 1
bias = 4 # b = 4

n = Neuron(weights, bias)

x = np.array([2,3]) # x1 = 2, x2 = 3

print(n.feed_forward(x)) # 0.9990889488055994

0.9990889488055994
